In [1]:
import importlib
import math
import torch
import os
import sys

import numpy as np
import pandas as pd
from datasets import Dataset
sys.path.append("../")

import src

importlib.reload(src)

from src.data_prep_utils import (  # noqa: E402
    conala_to_time_batches,
    load_time_sorted_conala,
    prep_for_hf,
)

importlib.reload(src.data_prep_utils)

from src.model import load_model_tok

importlib.reload(src.model)


from src.training import dd_analysis
importlib.reload(src.training)

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
MODEL = "CodeT5"
TRAIN_N = 300
BATCH_SIZE = 15
DECODER_LENGTH = 20
ENCODER_LENGTH = 15

TRAIN_ARGS = {
    "TRAIN_N": TRAIN_N,
    "BATCH_SIZE": BATCH_SIZE,
    "DECODER_LENGTH": DECODER_LENGTH,
    "ENCODER_LENGTH": ENCODER_LENGTH,
    "MODEL": MODEL,
    "SEQ_TRAINER_ARGS": {
        "output_dir": f"reports/{MODEL}_{TRAIN_N}_{BATCH_SIZE}/results",
        "overwrite_output_dir": True,
        "num_train_epochs": 2,
        "do_train": True,
        "do_eval": True,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 5e-4,
        "warmup_steps": 100,
        "weight_decay": 0.1,
        "label_smoothing_factor": 0.1,
        "predict_with_generate": True,
        "logging_dir": f"reports/{MODEL}_{TRAIN_N}_{BATCH_SIZE}/logs",
        "logging_steps": 100,
        "save_total_limit": 1,
        "save_strategy": "epoch",
        "logging_strategy": "epoch",
        "evaluation_strategy": "epoch",
        "load_best_model_at_end": False,
    },
}

In [ ]:
df = load_time_sorted_conala("../data/raw/conala")  # noqa: PD901
df = conala_to_time_batches(df, TRAIN_N, BATCH_SIZE)  # noqa: PD901

Unique questions: 2074
Sort Question IDs
Create t=0 training sample


In [ ]:
model, tokenizer = load_model_tok(MODEL, freeze=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cuda


In [ ]:
type(model)

transformers.models.t5.modeling_t5.T5ForConditionalGeneration

In [ ]:
print('a')

: 

In [ ]:
loss_df = dd_analysis(df, tokenizer=tokenizer, model=model, training_args=TRAIN_ARGS)

Map:   0%|          | 0/898 [00:00<?, ? examples/s]

INFO:root:Validation for time step 0


Map:   0%|          | 0/34 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,3.725700,4.085027,0.391200,0.157100,0.356800,0.360400,9.823500,0.055900,0.856900,0.866200,246,284
2,2.663600,4.104609,0.416300,0.170000,0.381300,0.383300,11.264700,0.078600,1.000000,1.031700,293,284


...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


INFO:absl:Using default tokenizer.
INFO:root:   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      4.303        0.294        0.079        0.251           0.252   

   eval_bleu  eval_gen_len  
0        0.0        12.028  
INFO:root:Validation for time step 52


Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


INFO:absl:Using default tokenizer.
INFO:root:   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      4.591        0.439          0.1        0.327           0.335   

   eval_bleu  eval_gen_len  
0      0.043        12.429  
